In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!cd '/content/drive/MyDrive/Colab Notebooks/dl projects/7'

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dl projects/7/spam (1).csv')

In [5]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [7]:
df.isnull().sum()

Category    0
Message     0
dtype: int64

In [8]:
df['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [9]:
print(df['Category'].value_counts()/len(df))

ham     0.865937
spam    0.134063
Name: Category, dtype: float64


In [10]:
df['label'] = [1 if x =='spam' else 0 for x in df['Category']]

In [11]:
df.head()

,Category,Message,label
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [12]:
!pip install tensorflow_text

In [13]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_text as text

In [14]:
df_spam = df[df['label'] == 1]

In [15]:
df_spam.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 747 entries, 2 to 5567
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  747 non-null    object
 1   Message   747 non-null    object
 2   label     747 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 23.3+ KB


In [16]:
df_ham = df[df['label']==0]

In [17]:
df_ham.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4825 entries, 0 to 5571
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  4825 non-null   object
 1   Message   4825 non-null   object
 2   label     4825 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 150.8+ KB


In [18]:
df_down_sample = df_ham.sample(len(df_spam))

In [19]:
df_balanced = pd.concat([df_down_sample, df_spam])

In [20]:
df_balanced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1494 entries, 388 to 5567
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  1494 non-null   object
 1   Message   1494 non-null   object
 2   label     1494 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 46.7+ KB


In [21]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_balanced['Message'],df_balanced['label'], test_size=.3,random_state=42)

In [22]:
x_train.shape, x_test.shape

((1045,), (449,))

In [23]:
y_train.shape, y_test.shape

((1045,), (449,))

In [24]:
bert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [25]:
from keras.src.layers.attention.multi_head_attention import regularization
keras.utils.set_random_seed(42)
text_input = tf.keras.layers.Input(shape = (), name='text', dtype = tf.string)
preprocessed_text = bert_preprocess(text_input)
output = bert_encoder(preprocessed_text)
l = tf.keras.layers.Dropout(0.1, name='dropout1')(output['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(l)
model = tf.keras.Model(inputs=[text_input], outputs=[l])

In [26]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_word_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_type_ids': (None,                                           
                              128)}                                                               
                                                                                              

In [27]:
METRICS  = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

In [28]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [29]:
model.fit(x_train, y_train, epochs=10,validation_data=(x_test, y_test))

Epoch 1/10
33/33 [==============================] - 35s 627ms/step - loss: 0.6525 - accuracy: 0.6325 - precision: 0.6334 - recall: 0.6039 - val_loss: 0.5665 - val_accuracy: 0.8129 - val_precision: 0.9568 - val_recall: 0.6681
Epoch 2/10
33/33 [==============================] - 16s 485ms/step - loss: 0.5162 - accuracy: 0.8201 - precision: 0.8357 - recall: 0.7903 - val_loss: 0.4644 - val_accuracy: 0.8775 - val_precision: 0.8798 - val_recall: 0.8836
Epoch 3/10
33/33 [==============================] - 16s 494ms/step - loss: 0.4455 - accuracy: 0.8498 - precision: 0.8365 - recall: 0.8641 - val_loss: 0.4073 - val_accuracy: 0.8820 - val_precision: 0.8510 - val_recall: 0.9353
Epoch 4/10
33/33 [==============================] - 17s 528ms/step - loss: 0.4112 - accuracy: 0.8469 - precision: 0.8571 - recall: 0.8272 - val_loss: 0.3723 - val_accuracy: 0.8842 - val_precision: 0.8600 - val_recall: 0.9267
Epoch 5/10
33/33 [==============================] - 17s 524ms/step - loss: 0.3636 - accuracy: 0.8785

In [30]:
pred = model.predict(x_test)

15/15 [==============================] - 5s 335ms/step


In [31]:
pred = [1 if x >=0.5 else 0 for x in pred]

In [32]:
pred[:10]

[1, 1, 1, 1, 1, 0, 0, 0, 1, 0]

In [33]:
pred = pd.DataFrame(pred)

In [34]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.95      0.82      0.88       217
           1       0.85      0.96      0.90       232

    accuracy                           0.90       449
   macro avg       0.90      0.89      0.89       449
weighted avg       0.90      0.90      0.89       449

